# Recoinize

[https://github.com/nicknochnack/FaceRecognition](https://github.com/nicknochnack/FaceRecognition/blob/main/Facial%20Verification%20with%20a%20Siamese%20Network%20-%20Final.ipynb)

# Setup

## Pull code from GitHub

In [2]:
%load_ext autoreload
%autoreload 2

from src.helpers.cmd import git_update, print_gpu_name
from src.helpers.install import install_requirements
# git_update("live-edit", force=False, show_output=False)
# install_requirements()

In [ ]:
!git fetch
!git pull

In [ ]:
print_gpu_name()

## Import dependencies

In [42]:
load_dataset_from = "HuggingFace" # "HuggingFace" or "Local"

In [41]:
# Import standard dependencies
import cv2
import os
import random
import numpy as np
from matplotlib import pyplot as plt


import tensorflow as tf
from datasets.load import load_dataset as HF_load_dataset

from src.helpers.load_data import load_data as local_load_data
from src.dataset import create_pairs

# Get and preprocess data

In [64]:
match load_dataset_from:
    case "HuggingFace":
        dataset = HF_load_dataset('photonsquid/coins-euro')
        # convert the dataset to tensorflow dataset
        # dataset = dataset.with_format('tf')
    case "Local":
        # load data from local files
        dataset = local_load_data("data/tests")
    case _:
        print("Invalid dataset source")

Using custom data configuration photonsquid--coins-euro-b75261052d3e19e0
Found cached dataset parquet (/home/chap/.cache/huggingface/datasets/photonsquid___parquet/photonsquid--coins-euro-b75261052d3e19e0/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
100%|██████████| 3/3 [00:00<00:00, 392.46it/s]


In [82]:
# split the dataset into train and test
# train_dataset = dataset['train']
# test_dataset = dataset['test']

# Rest

When data is a Tensorflow dataset, I can iterate over it with 

```python
tf_data_set = tf.data.Dataset.list_files("data/tests/train/*.png")
for image in data_set:
    print(image)
```
The type of `tf_data_set` is `ShuffleDataset` (I don't know why)

This object is an iterable (any TensorFlow Dataset: ShuffleDataset, OptionDataset, ... ).
So I cannot access to a specific element. I can only iterate over it:

```python
images = tf_data_set.as_numpy_iterator()
image = next(images)
print(image)
```

In [76]:
tf_data_set = tf.data.Dataset.list_files("data/tests/train/*.png")
for image in data_set:
    print(image)
    break

tf.Tensor(b'data/tests/train/ad_2euro_0.png', shape=(), dtype=string)


In [80]:
print(type(tf_data_set))
images = tf_data_set.as_numpy_iterator()
image = next(images)
print(image)

<class 'tensorflow.python.data.ops.dataset_ops.ShuffleDataset'>
b'data/tests/train/at_1cen_3.png'


When it is a HuggingFace dataset (`datasets.dataset_dict.DatasetDict`), I can iterate over it with 

```python
images = dataset['train']

for image in images:
      print(image)
```

In [69]:
# convert dataset into a numpy iterable

images = dataset['train']

for image in images:
      print(image)


{'image': <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=150x150 at 0x7FDF4A7CB490>, 'labels': {'value': '10cents', 'country': 'ad', 'edition': 'standard', 'variant': '0'}}
{'image': <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=150x150 at 0x7FDF4A7CA470>, 'labels': {'value': '10cents', 'country': 'ad', 'edition': 'standard', 'variant': '1'}}
{'image': <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=150x150 at 0x7FDF4A7CB130>, 'labels': {'value': '10cents', 'country': 'ad', 'edition': 'standard', 'variant': '2'}}
{'image': <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=150x150 at 0x7FDF4A7CAD40>, 'labels': {'value': '10cents', 'country': 'ad', 'edition': 'standard', 'variant': '3'}}
{'image': <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=150x150 at 0x7FDF4A7CA920>, 'labels': {'value': '10cents', 'country': 'ad', 'edition': 'standard', 'variant': '4'}}
{'image': <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=150x150 at 0x7FDF4A7CBFD0>, 'labels'

In [34]:
def create_twins(data):
    # here I have a dict with the following structure
    # data ={
    #    'image': tf.Tensor<content, Shape, Dtype>,
    #    'labels': {
    #       'value': tf.Tensor<content, Shape, Dtype>,
    #       'country': tf.Tensor<content, Shape, Dtype>,
    #       'edition': tf.Tensor<content, Shape, Dtype>,
    #       'vairant': tf.Tensor<content, Shape, Dtype>
    #     }
    # }
    # To get the value of a tensor, I need to use the numpy() method, 
    # and it needs to be converted from bytes

    # I want to create twins from this dataset.

    return ""


train_dataset.map(maping_dataset)

  0%|          | 0/1932 [00:00<?, ?ex/s]

{'value': <tf.Tensor: shape=(), dtype=string, numpy=b'10cents'>, 'country': <tf.Tensor: shape=(), dtype=string, numpy=b'ad'>, 'edition': <tf.Tensor: shape=(), dtype=string, numpy=b'standard'>, 'variant': <tf.Tensor: shape=(), dtype=string, numpy=b'0'>}


TypeError: Provided `function` which is applied to all elements of table returns a variable of type <class 'str'>. Make sure provided `function` returns a variable of type `dict` (or a pyarrow table) to update the dataset or `None` if you are only interested in side effects.

In [20]:
# create the pairs and labels for the training datasetmaping_dataset
train_dataset.map(maping_dataset)

# convert it to a tensorflow dataset
# train_dataset = tf.data.Dataset.from_tensor_slices(train_dataset)

  0%|          | 0/1932 [00:00<?, ?ex/s]

<class 'tensorflow.python.framework.ops.EagerTensor'>


TypeError: Provided `function` which is applied to all elements of table returns a variable of type <class 'str'>. Make sure provided `function` returns a variable of type `dict` (or a pyarrow table) to update the dataset or `None` if you are only interested in side effects.

In [58]:
# show first pair in subplot with plt
# plt.subplot(1, 2, 1)
# plt.imshow(train_dataset[0][0]["bytes"])
# plt.axis('off')
# plt.title("anchor")
# plt.subplot(1, 2, 2)
# plt.imshow(train_dataset[0][1]["bytes"])
# plt.axis('off')
# plt.title("positive" if sum(train_dataset[0][2]) == 5 else "negative")
# plt.rcParams['figure.facecolor'] = '#282A36'
# plt.rcParams['text.color'] = '#ecf0f1'
# plt.show()

# decode the next image
# convert dataset into a numpy iterator
train_dataset = train_dataset.as_numpy_iterator()
# decode the next image
image = next(train_dataset)
# convert the bytes to a numpy array
image = np.frombuffer(image[0]["bytes"], dtype=np.uint8)
# plot it with matplotlib
# convert the bytes to a numpy array
image = image.reshape((256, 256, 3))
# plot it
plt.imshow(image)
plt.show()


# plot it with matplotlib
# convert the bytes to a numpy array

# image = image.reshape((256, 256, 3))
# plot it
# plt.imshow(image)
# plt.show()

AttributeError: 'Dataset' object has no attribute 'as_numpy_iterator'

In [ ]:
# prepare the dataset for training
train_dataset = train_dataset.shuffle(1000)
train_dataset = train_dataset.batch(32)
train_dataset = train_dataset.prefetch(tf.data.experimental.AUTOTUNE)

# prepare the dataset for testing
test_dataset = test_dataset.shuffle(1000)
test_dataset = test_dataset.batch(32)
test_dataset = test_dataset.prefetch(tf.data.experimental.AUTOTUNE)